# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.sum(l_c ** 2 / l_s, axis=1) + \
                    np.sum(r_c ** 2 / r_s, axis=1)) / ((l_s + r_s)[0])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.nansum(np.log(l_c / l_s) * l_c, axis=1) + \
                 np.nansum(np.log(r_c / r_s) * r_c, axis=1)) / ((l_s + r_s)[0])

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)) / ((l_s + r_s)[0])

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):

        # Что делает этот блок кода?
        sorted_x, sorted_y = self.__sort_samples(x, y)  # получили отсортированный по возрастанию
        #x и соответсвующий y
        # class_number = np.unique(y).shape[0] #сколько разных классов есть в данном y
        # но это не сработает, когда при очередном разбиении выбыл какой-то непоследний класс.
        #Сработает для бинарной.

        # Что делает этот блок кода?
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # берем все, кроме первых и последних min_sample_split элементов, обеспечивая то,
        #что в вершине будет минимальное
        # заданное число вершин
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] \
        + (self.min_samples_split + 1)
        # r_border_ids те индексы, когда текущий элемент не равен следующему
        #сдвинутые вправо на (self.min_samples_split + 1)
        # то есть по факту границы элементов, когда класс поменялся

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # eq_el_count - массив, каждый элемент которого - количество подряд идущих одинаковых элементов
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        # one_hot_code поставили 1 в тех местах, в которых начинается новый класс
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # class_increments - матрица того же размера что и one_hot_code, и теперь вместо 1 на тех же местах
        # там будут стоять количества элементов, которые шли подряд
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split],
                                                                minlength=self.num_class)
        # class_increments[0] дописывает в первую строчку ещё и те количества элементов которые шли подряд
        # которые мы не учитывали при min_sample_split

        # Что делает этот блок кода?
        l_class_count = np.cumsum(class_increments, axis=0)
        # l_class_count для удобства, чтобы сразу понять, какие элементы пойдут налево при разбиении
        # по на данном индексе
        r_class_count = np.bincount(sorted_y, minlength=self.num_class) - l_class_count
        # а это элементы, которое пойдет вправо
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        # l_sizes количество элементов, которое пойдет влево
        r_sizes = sorted_y.shape[0] - l_sizes
        # r_sizes количество элементов, которое пойдет вправо

        # Что делает этот блок кода?
        # здесь мы высчитываем наименьший impurity
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # с left_el_id начинается правая часть выборки
        # возвращаем минимальное значение gs и порог по признаку
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):

        # создаем листовые вершины
        # если сейчас в узле объектов меньше чем в min_samples_split
        if (x.shape[0] <= self.min_samples_split):
            self.__create_leaf(y, node_id, depth)
            return
        elif (depth == self.max_depth):  # если достигнута максимальная глубина
            self.__create_leaf(y, node_id, depth)
            return
        elif np.any(np.unique(y, return_counts=True)[1] / y.shape[0] >= self.sufficient_share):
            # если какого то класса существенно больше
            self.__create_leaf(y, node_id, depth)
            return
        elif (np.unique(y).shape[0] == 1):
            # если уже только один класс
            self.__create_leaf(y, node_id, depth)
            return

        # это массивы для порогов, impurity
        f_size = x.shape[1]
        threshold = np.zeros(f_size)
        impurity = np.zeros(f_size)
        f_to_fit = self.get_feature_ids(f_size)
        for i in f_to_fit:
            impurity[i], threshold[i] = self.__find_threshold(x[:, i], y)

        # это пригодится для определения feature importance
        if self.G_function == self.__gini:
            my_purity = 1 - np.sum((np.unique(y, return_counts=True)[1] * 1. / y.shape[0]) ** 2)
        elif self.G_function == self.__misclass:
            my_purity = 1 - np.max(np.unique(y, return_counts=True)[1] * 1. / y.shape[0])
        elif self.G_function == self.__entropy:
            my_purity = -(np.nansum(
                np.log(np.unique(y, return_counts=True)[1] * 1. / y.shape[0]) \
                * np.unique(y, return_counts=True)[1] * 1. / y.shape[0], axis=1))

        # находим разбиение, соответствующее минимальному impurity
        f_min = impurity.argmin()

        # если разбить не удалось, создаем лист
        if np.isnan(threshold[f_min]):
            self.__create_leaf(y, node_id, depth)
            return

        # получаем разбиение
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, f_min, threshold[f_min])

        # если что-то имеет размер 0, то создаем лист
        if (x_left.shape[0] == 0 or x_right.shape[0] == 0):
            self.__create_leaf(y, node_id, depth)
            return

        # иначе создаем узел, в узле хранится тип вершины, индекс на котором произошло разбиение. 
        # порог, размер текущей выборки, impurity, и собственная I(S)
        self.tree[node_id] = (
        self.NON_LEAF_TYPE, f_min, threshold[f_min], x.shape[0], x.shape[1], impurity[f_min], my_purity)
        self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)

        # функция создания листа

    def __create_leaf(self, y, node_id, depth):
        counts = np.bincount(y, minlength=self.num_class)
        probs = counts.astype(float) / y.shape[0]
        self.tree[node_id] = (self.LEAF_TYPE, np.argmax(counts), probs, -1, -1, -1, -1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _, _, _, _ = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _, _, _, _ = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def feature_importances(self):
        imp = np.zeros(self.tree.values()[0][4])  # imp-importance, имеет размер = количеству признаков
        internal = sum([(1 - v[0]) for v in self.tree.itervalues()])  # считаем число узловых вершин
        for k, v in self.tree.items():
            if not v[0]:
                imp[v[1]] += (v[6] + v[5]) / internal  # действием в соответствии с формулой из лекции
        norm = np.sum(imp)  # перенормировка
        return imp * 1. / norm

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 2.52 ms, sys: 0 ns, total: 2.52 ms
Wall time: 2.98 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 8.44 ms, sys: 3.4 ms, total: 11.8 ms
Wall time: 15 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music',
              'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
idx = ((temp.wave >= 6) & (temp.wave <= 9))
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
                'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
idx = ((temp.wave >= 6) & (temp.wave <= 9))
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
                'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.1)

In [11]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [12]:
%time clf.fit(X_train, y_train)

CPU times: user 113 ms, sys: 377 µs, total: 113 ms
Wall time: 114 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
%time my_clf.fit(X_train, y_train.reshape(-1))

CPU times: user 1.33 s, sys: 0 ns, total: 1.33 s
Wall time: 1.33 s


## Проверка качества работы на Speed Dating Data

In [22]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5321637426900585

In [23]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5611218154570756

## Задание 3

In [24]:
def func_one():
    global df, X, y
    df = df.iloc[:, :97]
    df = df.drop(['id'], axis=1)
    df = df.drop(['idg'], axis=1)
    df = df.drop(['condtn'], axis=1)
    df = df.drop(['round'], axis=1)
    df = df.drop(['position', 'positin1'], axis=1)
    df = df.drop(['order'], axis=1)
    df = df.drop(['partner'], axis=1)
    df = df.drop(['age_o', 'race_o', 'pf_o_att',
                  'pf_o_sin', 'pf_o_int',
                  'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
                  'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
                 axis=1)
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = df.drop(['field'], axis=1)
    df = df.drop(['undergra'], axis=1)
    df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df = df.dropna(subset=['imprelig', 'imprace'])
    df = df.drop(['from', 'zipcode'], axis=1)
    df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = df.drop(['career'], axis=1)
    df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking',
                  'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts',
                  'music', 'shopping', 'yoga'], axis=1)
    df = df.drop(['expnum'], axis=1)
    feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
    idx = ((temp.wave >= 6) & (temp.wave <= 9))
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                            'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
        (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                    'amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
    feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
    idx = ((temp.wave >= 6) & (temp.wave <= 9))
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
        (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                    'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))

        df = df.drop(feat, axis=1)
    df = df.drop(['wave'], axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
        .drop(['gender'], axis=1) \
        .dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
        .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
        .dropna()

    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'),
                           on='pid',
                           how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values


def func_two():
    global X, y, X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.1)


In [25]:
from sklearn.pipeline import Pipeline

df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')
X = np.empty(0)
y = np.empty(0)
X_train = np.empty(0)
X_test = np.empty(0)
y_train = np.empty(0)
y_test = np.empty(0)
pipeline1 = Pipeline(
    [('first', func_one()), ('second', func_two()), ('third', MyDecisionTreeClassifier(min_samples_split=2))])
df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')
pipeline2 = Pipeline(
    [('first', func_one()), ('second', func_two()), ('third', DecisionTreeClassifier(min_samples_split=2))])


In [26]:
my_clf_pipe = pipeline1.fit(X_train, y_train)

In [27]:
clf_pipe = pipeline2.fit(X_train, y_train)

In [28]:
f1_score(y_pred=my_clf_pipe.predict(X_test), y_true=y_test, average='macro')

0.5186003484758671

In [29]:
f1_score(y_pred=clf_pipe.predict(X_test), y_true=y_test, average='macro')

0.5294117647058824

## Задание 4

In [30]:
names = df_pair.columns[1:]

In [31]:
pd.Series(index=names, data=my_clf.feature_importances()).sort_values(ascending=False)[:10]

int_corr      0.053160
field_cd      0.042527
imprace_f     0.041369
age_f         0.036940
age           0.036593
samerace      0.036132
race          0.034012
fun1_1_f      0.031843
go_out_f      0.030052
intel1_1_f    0.029736
dtype: float64

In [32]:
pd.Series(index=names, data=clf.feature_importances_).sort_values(ascending=False)[:10]

int_corr      0.068601
amb1_1_f      0.048948
age           0.036049
age_f         0.034163
career_c_f    0.030970
sinc1_1       0.029565
attr3_1       0.028514
attr3_1_f     0.024953
shar1_1       0.022555
attr1_1_f     0.021998
dtype: float64

## Задание 5

In [33]:
classifier = RandomForestClassifier()

In [34]:
params_grid = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': range(10, 21),
    'max_features': ['auto', 'log2', None]
}

In [35]:
grid_cv = GridSearchCV(classifier, params_grid)

In [36]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'max_features': ['auto', 'log2', None], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [37]:
grid_cv.best_params_

{'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 16}

In [38]:
grid_cv.best_score_

0.8405112531258683